# Sensors

### Install packages

In [1]:
#r "nuget:Microsoft.ML,1.4.0"
using XPlot.Plotly;
using System;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;

Installing package Microsoft.ML, version 1.4.0.........

### Declare data models

In [2]:
public class ModelInput
{
    [ColumnName("Temperature"), LoadColumn(0)]
    public float Temperature { get; set; }

    [ColumnName("Luminosity"), LoadColumn(1)]
    public float Luminosity { get; set; }

    [ColumnName("Infrared"), LoadColumn(2)]
    public float Infrared { get; set; }

    [ColumnName("Distance"), LoadColumn(3)]
    public float Distance { get; set; }

    [ColumnName("CreatedAt"), LoadColumn(4)]
    public string CreatedAt { get; set; }

    [ColumnName("Label"), LoadColumn(5)]
    public string Source { get; set; }
}

public class ModelOutput
{
    [ColumnName("PredictedLabel")]
    public string PredictedLabel;

    [ColumnName("Score")]
    public float[] Score;
}

### Instantiate the ML context

In [3]:
private static readonly MLContext mlContext = new MLContext(2020);

### Load data

In [4]:
private const string DATASET_PATH = "./sensors_data.csv";
IDataView data = mlContext.Data.LoadFromTextFile<ModelInput>(
    path: DATASET_PATH,
    hasHeader: true,
    separatorChar: ',');

### Shuffle and split data to train and test by a fraction of 0.2

In [5]:
var shuffledData = mlContext.Data.ShuffleRows(data, seed: 2020);
var split = mlContext.Data.TrainTestSplit(shuffledData, testFraction: 0.2);
var trainingData = split.TrainSet;
var testingData = split.TestSet;

In [6]:
var features = mlContext.Data.CreateEnumerable<ModelInput>(trainingData, true);
display(features.Take(10)); // be carefull here not to display the entire dataset

index,Temperature,Luminosity,Infrared,Distance,CreatedAt,Source
0,32.18,6.84,0,28.69,01/03/2020 10:31:27,Day
1,115,100,43.85,195.19,01/03/2020 10:26:04,Lighter
2,30.89,73.44,0,24.32,01/03/2020 10:33:01,FlashLight
3,81.48,100,93.46,400,01/03/2020 10:27:54,Lighter
4,32.18,6.64,0,76.97,01/03/2020 10:31:06,Day
5,24.28,14.45,0,17.49,01/03/2020 10:23:42,FlashLight
6,27.34,70.02,0,193.76,01/03/2020 10:25:27,FlashLight
7,23.48,87.6,0,343.57,01/03/2020 10:22:23,FlashLight
8,34.11,100,92.38,7.16,01/03/2020 10:28:07,Lighter
9,31.86,57.52,0,400,01/03/2020 10:33:25,FlashLight


### Categories

In [7]:
var categories = trainingData.GetColumn<string>("Label");
var categoriesHistogram = Chart.Plot(
    new Graph.Histogram 
    {
        x = categories
    }
);
display(categoriesHistogram);

### Plot diagram

In [8]:
var featuresTemperatures = features.Select(f => f.Temperature);
var featuresLuminosities = features.Select(f => f.Luminosity);
var featuresInfrareds = features.Select(f => f.Infrared);
var featuresDistances = features.Select(f => f.Distance);

var categoriesDiagram = Chart.Plot(new[] {
    new Graph.Box { y = featuresTemperatures, name = "Temperature" },
    new Graph.Box { y = featuresLuminosities, name = "Luminosity" },
    new Graph.Box { y = featuresInfrareds, name = "Infrared" },
    new Graph.Box { y = featuresDistances, name = "Distance" }
});
display(categoriesDiagram);

In [9]:
#r "nuget:MathNet.Numerics, 4.9.0"
using MathNet.Numerics.Statistics;

Installing package Microsoft.ML, version 1.4.0.........

Installed package MathNet.Numerics version 4.9.0

In [10]:
var featureColumns = new string[] { "Temperature", "Luminosity", "Infrared", "Distance" };

var correlationMatrix = new List<List<double>>();
correlationMatrix.Add(featuresTemperatures.Select(x => (double)x).ToList());
correlationMatrix.Add(featuresLuminosities.Select(x => (double)x).ToList());
correlationMatrix.Add(featuresInfrareds.Select(x => (double)x).ToList());
correlationMatrix.Add(featuresDistances.Select(x => (double)x).ToList());

var length = featureColumns.Length;

var z = new double[length, length];
for (int x = 0; x < length; ++x)
{
    for (int y = 0; y < length - 1 - x; ++y)
    {
        var seriesA = correlationMatrix[x];
        var seriesB = correlationMatrix[length - 1 - y];

        var value = Correlation.Pearson(seriesA, seriesB);

        z[x, y] = value;
        z[length - 1 - y, length - 1 - x] = value;
    }

    z[x, length - 1 - x] = 1;
}

var correlationMatrixHeatmap = Chart.Plot(
    new Graph.Heatmap 
    {
        x = featureColumns,
        y = featureColumns.Reverse(),
        z = z,
        zmin = -1,
        zmax = 1
    }
);
display(correlationMatrixHeatmap);

### Concatenate features

In [11]:
var preprocessingPipeline = mlContext.Transforms.Conversion.MapValueToKey("Label")
    .Append(mlContext.Transforms.Concatenate("Features", featureColumns))
    .Append(mlContext.Transforms.NormalizeMinMax("Features", "Features"));